<a href="https://colab.research.google.com/github/dgmyaing/labs/blob/main/HedonicPricingFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
#Importing anything I may need to use

url = 'https://raw.githubusercontent.com/DS3001/labs/refs/heads/main/04_hedonic_pricing/airbnb_hw.csv'
df = pd.read_csv(url)

In [3]:
url = 'https://raw.githubusercontent.com/DS3001/labs/refs/heads/main/04_hedonic_pricing/airbnb_hw.csv'
data = pd.read_csv(url)

print("Summary of Data:")
data.info()

print("\nFirst Few Rows:")
print(data.head())
print("\nMissing Values:")
print(data.isnull().sum())
data.dropna(inplace=True)

data.drop_duplicates(inplace=True) #Getting rid of duplication

print("\nData Stats Description:")
print(data.describe())

# I feel like this should be a good enough introduction to most of the data present

Summary of Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30478 entries, 0 to 30477
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Host Id                     30478 non-null  int64  
 1   Host Since                  30475 non-null  object 
 2   Name                        30478 non-null  object 
 3   Neighbourhood               30478 non-null  object 
 4   Property Type               30475 non-null  object 
 5   Review Scores Rating (bin)  22155 non-null  float64
 6   Room Type                   30478 non-null  object 
 7   Zipcode                     30344 non-null  float64
 8   Beds                        30393 non-null  float64
 9   Number of Records           30478 non-null  int64  
 10  Number Of Reviews           30478 non-null  int64  
 11  Price                       30478 non-null  object 
 12  Review Scores Rating        22155 non-null  float64
dtypes: float64(4),

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

features = ['Property Type', 'Room Type', 'Beds', 'Number of Records', 'Number Of Reviews', 'Review Scores Rating']
X = data[features]
y = data['Price']

#Adding relevant data points into a consolidated place

In [ ]:
# Converting price into numeric datapoints
data['Price'] = data['Price'].replace(',', '', regex=True).astype(float)
X = data[features]
y = data['Price']
X = pd.get_dummies(X, columns=['Property Type', 'Room Type'], drop_first=True)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #80/20 test split

simple_model = LinearRegression()
simple_model.fit(X_train, y_train)

y_train_pred_simple = simple_model.predict(X_train)
y_test_pred_simple = simple_model.predict(X_test)


rmse_train_simple = mean_squared_error(y_train, y_train_pred_simple, squared=False)
rmse_test_simple = mean_squared_error(y_test, y_test_pred_simple, squared=False)
r2_train_simple = r2_score(y_train, y_train_pred_simple)
r2_test_simple = r2_score(y_test, y_test_pred_simple)
#Getting the RMSE and R^2
print("Results")
print(f"Training RMSE: {rmse_train_simple:.2f}, Training R^2: {r2_train_simple:.2f}")
print(f"Test RMSE: {rmse_test_simple:.2f}, Test R^2: {r2_test_simple:.2f}")
#unsure of why I'm getting the FutureWarning, can't figure out how to get past that

Simple Model Results:
Training RMSE: 130.69, Training R^2: 0.22
Test RMSE: 133.11, Test R^2: 0.25


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:

X_train['Beds_Squared'] = X_train['Beds'] ** 2
X_test['Beds_Squared'] = X_test['Beds'] ** 2

complex_model = LinearRegression()
complex_model.fit(X_train, y_train)

y_train_pred_complex = complex_model.predict(X_train)
y_test_pred_complex = complex_model.predict(X_test)

rmse_train_complex = mean_squared_error(y_train, y_train_pred_complex, squared=False)
rmse_test_complex = mean_squared_error(y_test, y_test_pred_complex, squared=False)
r2_train_complex = r2_score(y_train, y_train_pred_complex)
r2_test_complex = r2_score(y_test, y_test_pred_complex)

print("\nComplex Model Results:")
print(f"Training RMSE: {rmse_train_complex:.2f}, Training R^2: {r2_train_complex:.2f}")
print(f"Test RMSE: {rmse_test_complex:.2f}, Test R^2: {r2_test_complex:.2f}")



Complex Model Results:
Training RMSE: 130.68, Training R^2: 0.22
Test RMSE: 133.45, Test R^2: 0.25


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


The R² values are pretty low—0.22 for training and 0.25 for testing—which means the models aren’t doing a great job of explaining the variance in Price. This suggests the models might be too basic to capture the relationships in the data, or we could be missing some important features. That said, since the training and test set performances are similar, there’s no obvious sign of overfitting. If the models were overfitting, we’d usually see much higher R² and lower RMSE for the training set compared to the test set.

I started by exploring and cleaning the dataset to prepare it for regression analysis. This involved handling missing values, transforming categorical variables, and cleaning up numeric columns that had special characters like commas. My key takeaway from this step was that cleaning and preprocessing are essential to make the data usable, especially when working with a combination of numerical and categorical variables.

After cleaning the data, I built a simple linear regression model without adding any transformations or interaction terms. The results showed a training RMSE of 130.69 and a training R² of 0.22, while the test RMSE was 133.11 and the test R² was 0.25. These low R² values indicated that the model couldn’t explain much of the variance in the target variable (price), which suggested it was underfitting. The simplicity of the model likely prevented it from capturing the complexity of the data.

To improve the model, I constructed a more complex version by including interaction terms and transformations to better capture relationships between features. However, the results were nearly identical to the simple model: a training RMSE of 130.68, training R² of 0.22, test RMSE of 133.45, and test R² of 0.25. This showed that the added complexity didn’t improve the model’s performance, meaning the additional terms failed to uncover any new patterns. The model still appeared to be underfitting.

When comparing the two models, I noticed their similar performance on both the training and test sets. This ruled out overfitting since an overfit model would have performed much better on the training set than the test set. The low R² values for both models confirmed that underfitting was the real issue, likely due to missing key predictors or insufficient feature engineering.

To address this, I considered using a Lasso regression model. Lasso regularization could help identify the most important features by shrinking less relevant ones to zero, which would improve interpretability and reduce noise. While regularization could help prevent overfitting, I realized it wouldn’t fully resolve the underfitting problem if critical features were missing. This emphasized the need for careful feature selection and engineering in addition to model tuning.
